<a href="https://colab.research.google.com/github/bhatzahidmajeed/breast_cancer_ml/blob/zahid/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost:
XGBoost, short for "Extreme Gradient Boosting," is a powerful and widely-used machine learning algorithm renowned for its superior performance in various data-driven tasks, particularly in structured/tabular data scenarios.
- It belongs to the gradient boosting family, which ensemble weak learners (usually decision trees) to create a robust and accurate predictive model.
- XGBoost's key strength lies in its ability to optimize both computational efficiency and model accuracy simultaneously.
- It achieves this by employing regularization techniques like L1 and L2 regularization to prevent overfitting and by utilizing a clever implementation that takes advantage of parallel processing and tree-pruning.

XGBoost, when used carefully and with proper considerations, can be a valuable tool for breast cancer detection, providing accurate and reliable predictions that can aid in early diagnosis and treatment decisions.

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, balanced_accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('breast-cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [4]:
df.drop('id', axis=1, inplace=True)
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int)
# Encode M (Malignant) as 1 & B (Benign) as 0.
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Training the Model:
First, we create a function that can help us iterate our models faster.

In [5]:
def train_evaluate_model(model, X_train, y_train, X_test,y_test):
    model.fit(X_train, y_train)  # Fit the model instance
    predictions = model.predict(X_test) # Calculate predictions
    # Compute metrics for evaluation
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    precision = precision_score(y_test, predictions, zero_division = 1)
    recall = recall_score(y_test, predictions)
    balanced_accuracy = balanced_accuracy_score(y_test, predictions)
    # Create a dataframe to visualize the results
    eval_df = pd.DataFrame([[accuracy, f1, precision, recall, balanced_accuracy]], columns=['Accuracy', 'F1-Score', 'Precision', 'Recall', 'Balanced Accuracy'])
    return eval_df

Then, we evaluate the model, without feature selection & later with various methods of feature selection.

### Case I: With All Features:

In [6]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data
xg = xgb.XGBClassifier()
results = train_evaluate_model(xg, X_train, y_train, X_test, y_test)

In [7]:
results.index = ['XGBoost (All Features)']
results.sort_values(by='F1-Score',ascending=False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
XGBoost (All Features),0.956140,0.941176,0.952381,0.930233,0.951032


### Case II: Manual Feature Selection:

In [8]:
# Get the absolute value of the correlation:
corr = df.corr()
cor_target = abs(corr["diagnosis"])
cor_target

diagnosis                  1.000000
radius_mean                0.730029
texture_mean               0.415185
perimeter_mean             0.742636
area_mean                  0.708984
smoothness_mean            0.358560
compactness_mean           0.596534
concavity_mean             0.696360
concave points_mean        0.776614
symmetry_mean              0.330499
fractal_dimension_mean     0.012838
radius_se                  0.567134
texture_se                 0.008303
perimeter_se               0.556141
area_se                    0.548236
smoothness_se              0.067016
compactness_se             0.292999
concavity_se               0.253730
concave points_se          0.408042
symmetry_se                0.006522
fractal_dimension_se       0.077972
radius_worst               0.776454
texture_worst              0.456903
perimeter_worst            0.782914
area_worst                 0.733825
smoothness_worst           0.421465
compactness_worst          0.590998
concavity_worst            0

In [9]:
best_threshold = 0
best_f1_score = 0
best_names = None  # Initialize best_names to None

for threshold in np.arange(0, 1, 0.005):  # This will iterate from 0 to 1 with a step of 0.05
    # Select highly correlated features based on the current threshold
    relevant_features = cor_target[cor_target > threshold]
    names = [index for index, value in relevant_features.iteritems()]
    names.remove('diagnosis')  # Drop the target variable from the results

    # Check if there are any relevant features
    if not names:
        continue  # Skip this iteration and try the next threshold

    X = df[names]
    y = df['diagnosis']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()  # Create an instance of standard scaler
    scaler.fit(X_train)  # Fit it to the training data
    X_train = scaler.transform(X_train)  # Transform training data
    X_test = scaler.transform(X_test)  # Transform validation data

    xg = xgb.XGBClassifier()

    # Train the model
    xg.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = xg.predict(X_test)

    # Calculate the F1 score
    current_f1_score = f1_score(y_test, y_pred)

    # Check if the current F1 score is better than the best F1 score so far
    if current_f1_score > best_f1_score:
        best_f1_score = current_f1_score
        best_threshold = threshold
        best_names = names  # Update the best_names with the current relevant features

# Check if any relevant features were found before proceeding
if best_names is None:
    print("No relevant features found with any threshold. Please adjust the threshold.")
else:
    print("Best Threshold:", best_threshold)
    print("Best F1 Score:", best_f1_score)
    print("Best Relevant Features:", best_names)
    print('No. of Best Relevant Features:', len(best_names))

Best Threshold: 0.325
Best F1 Score: 0.9523809523809524
Best Relevant Features: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'radius_se', 'perimeter_se', 'area_se', 'concave points_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst']
No. of Best Relevant Features: 22


In [10]:
corr = df.corr()
cor_target = abs(corr["diagnosis"])
# Select highly correlated features:
# (Please put the threshold value manually)
relevant_features = cor_target[cor_target > 0.01]
# Collect the names of the features:
names = [index for index, value in relevant_features.iteritems()]
# Drop the target variable from the results:
names.remove('diagnosis')
len(names)

28

In [11]:
X = df[names]
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data
xg = xgb.XGBClassifier()
mfs = train_evaluate_model(xg, X_train, y_train, X_test, y_test)

In [12]:
mfs.index = ['XGBoost (Manual)']
results = results.append(mfs)
results.sort_values(by = 'F1-Score', ascending = False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
XGBoost (All Features),0.956140,0.941176,0.952381,0.930233,0.951032
XGBoost (Manual),0.956140,0.941176,0.952381,0.930233,0.951032


### Case III: L1 Regularization:
XGBoost uses L1 regularization as part of its split finding process during tree construction.

In XGBoost, the regularization term consists of two parts: L1 regularization (Lasso regularization) and L2 regularization (Ridge regularization). The objective function being minimized during the training process is a combination of the loss function and the regularization term.

The L1 regularization term (Lasso) encourages sparsity in the feature space by adding the absolute values of the weights (coefficients) of the individual features to the objective function. This results in some features having exactly zero weights, effectively excluding them from the model, thus performing feature selection.

In [13]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']
data_dmatrix = xgb.DMatrix(data=X, label=y)

# Set L1 regularization (alpha)
alpha = 0.1  # Set your desired value for L1 regularization

# Create and configure the XGBoost model
xgboost_model = xgb.train(params={'objective': 'binary:logistic', 'alpha': alpha},
                          dtrain=data_dmatrix, num_boost_round=100)

# Get feature importances
feature_importances = xgboost_model.get_score(importance_type='weight')

# Sort feature importances in descending order to get the most important features first
sorted_feature_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# Print the names of the most important features
print("Most important features:")
for feature, importance in sorted_feature_importances:
    print(f"{feature}: {importance}")

Most important features:
area_worst: 34.0
area_se: 33.0
texture_worst: 30.0
texture_mean: 22.0
concave points_worst: 19.0
compactness_se: 18.0
concave points_mean: 17.0
smoothness_worst: 16.0
concavity_worst: 15.0
smoothness_mean: 14.0
symmetry_worst: 11.0
symmetry_se: 9.0
concave points_se: 8.0
fractal_dimension_se: 8.0
perimeter_worst: 8.0
symmetry_mean: 6.0
smoothness_se: 5.0
area_mean: 4.0
concavity_mean: 4.0
fractal_dimension_mean: 4.0
radius_worst: 4.0
fractal_dimension_worst: 4.0
compactness_mean: 3.0
radius_mean: 2.0
radius_se: 2.0
perimeter_se: 2.0
concavity_se: 2.0
compactness_worst: 1.0


In [14]:
threshold = 10  # Set your desired threshold value

# Filter feature importances based on the threshold
important_features = {feature: importance for feature, importance in feature_importances.items() if importance > threshold}

In [15]:
X = df[important_features]
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data
xg = xgb.XGBClassifier()
l1 = train_evaluate_model(xg, X_train, y_train, X_test, y_test)

In [16]:
l1.index = ['XGBoost (L1)']
results = results.append(l1)
results.sort_values(by = 'F1-Score', ascending = False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

,Accuracy,F1-Score,Precision,Recall,Balanced Accuracy
XGBoost (L1),0.964912,0.952381,0.975610,0.930233,0.958074
XGBoost (All Features),0.956140,0.941176,0.952381,0.930233,0.951032
XGBoost (Manual),0.956140,0.941176,0.952381,0.930233,0.951032


### Case IV: Variance Thresholding:
This method removes features with low variance, assuming that features with little variation provide limited information for the model.

In [ ]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# Initialize an array of threshold values to try
threshold_values = np.arange(0.1, 1.0, 0.005)

best_threshold = None
best_f1_score = 0.0

for threshold_value in threshold_values:
    # Perform variance thresholding
    variance_threshold = VarianceThreshold(threshold=threshold_value)
    X_high_variance = variance_threshold.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_high_variance, y, test_size=0.2, random_state=42)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the logistic regression model
    xg = xgb.XGBClassifier()
    xg.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = xg.predict(X_test)

    # Calculate the F1 score
    f1 = f1_score(y_test, y_pred)

    # Check if this threshold gives a higher F1 score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = threshold_value

print("Best Threshold Value:", best_threshold)
print("Best F1 Score:", best_f1_score)

In [ ]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']
# (Please put the threshold value manually)
variance_threshold = VarianceThreshold(threshold=0.30500000000000016)
# Fit the VarianceThreshold object to the data.
X_high_variance = variance_threshold.fit_transform(X)
# The "get_support" method returns a boolean mask of the selected features.
selected_features_mask = variance_threshold.get_support()
# You can use this mask to get the indices or names of the selected features.
selected_feature_indices = np.where(selected_features_mask)[0]
selected_feature_names = X.columns[selected_feature_indices]
selected_feature_names

In [ ]:
X = df[selected_feature_names]
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
scaler = StandardScaler() # Create an instance of standard scaler
scaler.fit(X_train) # Fit it to the training data
X_train = scaler.transform(X_train) # Transform training data
X_test = scaler.transform(X_test) # Transform validation data
xg = xgb.XGBClassifier()
vt = train_evaluate_model(xg, X_train, y_train, X_test, y_test)

In [ ]:
vt.index = ['XGBoost (VT)']
results = results.append(vt)
results.sort_values(by = 'F1-Score', ascending = False).style.background_gradient(cmap = sns.color_palette("ch:s=-.2,r=.6", as_cmap=True))

## Stratified K-Fold:
In binary classification problems, an imbalanced dataset occurs when one class (the minority class) has significantly fewer samples compared to the other class (the majority class).

In [ ]:
px.histogram(data_frame=df, x='diagnosis', color='diagnosis',color_discrete_sequence=['red','blue'])

Cross-validation is a technique that helps estimate the model's generalization ability by dividing the data into multiple folds and training/evaluating the model on different subsets of the data. Stratified K-Fold ensures that each fold maintains the same class distribution as the original dataset, which is particularly important for imbalanced datasets.

In [ ]:
X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

# Define the number of folds for cross-validation
n_splits = 5  # Set the desired number of folds

# Create the StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# Create an empty list to store the accuracy scores for each fold
accuracy_scores = []

# Loop through each fold and train/evaluate the logistic regression model
for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create and train the logistic regression model
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy for this fold and store it in the list
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Calculate the mean and standard deviation of the accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

# Print the mean and standard deviation of accuracy
print("Mean Accuracy:", mean_accuracy*100, '%')
print("Standard Deviation of Accuracy:", std_accuracy)